# Trabajo Práctico Integrador - Modelo BETO (BERT en español)

Este trabajo desarrolla un modelo para la evaluación y corrección de respuestas abiertas en exámenes, comparando las respuestas de los estudiantes con las generadas por un modelo basado en GPT-4.
El objetivo es determinar la similitud entre ambas respuestas para asignar una calificación adecuada, optimizando así la evaluación de preguntas abiertas.

In [1]:
# Instalación de las librerías necesarias
!pip install transformers
!pip install sentencepiece  # Necesario para algunos tokenizadores

In [2]:
# Cargamos el modelo BETO desde HuggingFace
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")
bert_model = AutoModel.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/364 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertModel were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
## Carga de Datos

import pandas as pd
import torch
from google.colab import files

# uploaded = files.upload()

data = pd.read_json("https://raw.githubusercontent.com/SolLopreste/TPI-ModeloCorreccionExamenes/refs/heads/main/data/Dataset.json?token=GHSAT0AAAAAACU5TJDJOOIN5TAP2M7QLG6YZ2HPUPA")

# Eliminar filas que tengan valores vacíos en las columnas importantes
data = data.dropna(subset=["Pregunta", "Respuesta Correcta", "Respuesta Alumno"])

data.head()

# data.shape

HTTPError: HTTP Error 404: Not Found

In [ ]:
### Dataset

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

class ExamDataset(Dataset):
    def __init__(self, data):
        self.data = data.reset_index(drop=True)
        # Convertir los puntajes a valores numéricos continuos entre 0 y 1
        self.data['Puntaje_Normalizado'] = self.data['Puntaje'] / 10.0  # Normalizar entre 0 y 1

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        pregunta = item["Pregunta"]
        respuesta_correcta = item["Respuesta Correcta"]
        respuesta_estudiante = item["Respuesta Alumno"]
        puntaje = item["Puntaje_Normalizado"]  # Valor normalizado entre 0 y 1

        return {
            'pregunta': pregunta,
            'respuesta_correcta': respuesta_correcta,
            'respuesta_estudiante': respuesta_estudiante,
            'puntaje': torch.tensor(puntaje, dtype=torch.float)
        }

In [ ]:
# Dataloader

from sklearn.model_selection import train_test_split

# Función para preparar los lotes
def collate_batch(batch):
    preguntas = [item['pregunta'] for item in batch]
    respuestas_correctas = [item['respuesta_correcta'] for item in batch]
    respuestas_estudiantes = [item['respuesta_estudiante'] for item in batch]
    puntajes = torch.stack([item['puntaje'] for item in batch])  # Tensor de puntajes

    # Usar el tokenizador con text y text_pair
    inputs = tokenizer(
        text=respuestas_correctas,
        text_pair=respuestas_estudiantes,
        padding='longest',
        truncation=True,
        return_tensors='pt'
    )

    return inputs, puntajes

# Crear los DataLoaders de entrenamiento y prueba
def create_dataloaders(data, batch_size=8, test_size=0.2):
    # Dividir los datos en conjuntos de entrenamiento y prueba
    train_data, test_data = train_test_split(data, test_size=test_size, random_state=42)

    # Crear el dataset de entrenamiento y prueba
    train_dataset = ExamDataset(train_data)
    test_dataset = ExamDataset(test_data)

    # Crear los DataLoaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

    return train_dataloader, test_dataloader

# Crear los DataLoaders
train_dataloader, test_dataloader = create_dataloaders(data)

In [ ]:
## Definir el Modelo de Regresión

import torch.nn as nn
from transformers import AutoModel

class BertRegressionModel(nn.Module):
    def __init__(self, bert_model):
        super(BertRegressionModel, self).__init__()
        self.bert = bert_model
        self.dropout = nn.Dropout(p=0.3)
        self.regressor = nn.Linear(self.bert.config.hidden_size, 1)

    def forward(self, input_ids, attention_mask, token_type_ids):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        pooled_output = outputs.pooler_output  # Tamaño [batch_size, hidden_size]
        pooled_output = self.dropout(pooled_output)
        output = self.regressor(pooled_output)  # Tamaño [batch_size, 1]
        return output.squeeze(-1)  # Tamaño [batch_size]

In [ ]:
# Instanciar el modelo
model = BertRegressionModel(bert_model)

# Mover el modelo al dispositivo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
### Definir el Optimizador y la Función de Pérdida

from transformers import AdamW, get_linear_schedule_with_warmup

# Hiperparámetros
epochs = 20
learning_rate = 2e-5

# Definir el optimizador
optimizer = AdamW(model.parameters(), lr=learning_rate)

# Función de pérdida
loss_fn = nn.MSELoss()

# Scheduler para ajustar la tasa de aprendizaje
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


In [ ]:
## Entrenamiento del Modelo

for epoch in range(epochs):
    print(f"======== Época {epoch + 1} / {epochs} ========")
    model.train()
    total_train_loss = 0

    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs, targets = batch
        inputs = {key: value.to(device) for key, value in inputs.items()}
        targets = targets.to(device)

        outputs = model(**inputs)
        loss = loss_fn(outputs, targets)

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_train_loss += loss.item()

    avg_train_loss = total_train_loss / len(train_dataloader)
    print(f"Pérdida promedio de entrenamiento: {avg_train_loss:.4f}")

    # Evaluación en el conjunto de prueba
    model.eval()
    total_eval_loss = 0
    predictions = []
    true_values = []

    with torch.no_grad():
        for batch in test_dataloader:
            inputs, targets = batch
            inputs = {key: value.to(device) for key, value in inputs.items()}
            targets = targets.to(device)

            outputs = model(**inputs)
            loss = loss_fn(outputs, targets)

            total_eval_loss += loss.item()

            predictions.extend(outputs.detach().cpu().numpy())
            true_values.extend(targets.cpu().numpy())

    avg_val_loss = total_eval_loss / len(test_dataloader)
    print(f"Pérdida promedio de validación: {avg_val_loss:.4f}")

    # Calcular métricas de evaluación
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

    mse = mean_squared_error(true_values, predictions)
    mae = mean_absolute_error(true_values, predictions)
    r2 = r2_score(true_values, predictions)

    print(f"MSE: {mse:.4f}, MAE: {mae:.4f}, R2: {r2:.4f}")

print("Entrenamiento completo!")

In [ ]:
## Evaluación Final y Visualización de Resultados

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Convertir las listas a arrays numpy
predictions = np.array(predictions)
true_values = np.array(true_values)

# Desnormalizar los puntajes para obtener valores entre 0 y 10
predictions_desnormalizadas = predictions * 10.0
true_values_desnormalizadas = true_values * 10.0

# Calcular métricas con puntajes desnormalizados
mse = mean_squared_error(true_values_desnormalizadas, predictions_desnormalizadas)
mae = mean_absolute_error(true_values_desnormalizadas, predictions_desnormalizadas)
r2 = r2_score(true_values_desnormalizadas, predictions_desnormalizadas)

print(f"Métricas finales con puntajes desnormalizados:")
print(f"MSE: {mse:.2f}, MAE: {mae:.2f}, R2: {r2:.4f}")

# Visualizar resultados
plt.figure(figsize=(8,6))
sns.scatterplot(x=true_values_desnormalizadas, y=predictions_desnormalizadas)
plt.xlabel('Puntajes Reales')
plt.ylabel('Puntajes Predichos')
plt.title('Puntajes Reales vs Predichos')
plt.show()

# Mostrar distribución de errores
errors = true_values_desnormalizadas - predictions_desnormalizadas
plt.figure(figsize=(8,6))
sns.histplot(errors, bins=20, kde=True)
plt.xlabel('Error de Predicción')
plt.title('Distribución de los Errores de Predicción')
plt.show()


In [ ]:
# Guardar el tokenizer
tokenizer.save_pretrained('modelo_beto_regresion')

# Guardar el modelo BERT interno
model.bert.save_pretrained('modelo_beto_regresion')

# Guardar el estado del modelo personalizado
torch.save(model.state_dict(), 'modelo_beto_regresion/bert_regression_model.pt')


In [ ]:
from google.colab import files
files.download('modelo_beto_regresion/bert_regression_model.pt')